In [1]:
import numpy
import pandas
import pandas_profiling
import matplotlib.pyplot as plt
import gymnasium
from gymnasium.wrappers import RecordVideo
from gymnasium.utils.play import play
from gymnasium.experimental.wrappers import GrayscaleObservationV0

In [2]:
#  sets the precision to 3 decimal places.
numpy.set_printoptions(precision=3)

# Enviroment Setup

## Game Mode (Mode 2 is the real game)

Game Mode 0 (default) - 15

Changing the Game Mode (Good For Testing Generalisation of the model) 

In [3]:
mode = 2

In [4]:

env = gymnasium.make("ALE/SpaceInvaders-v5", render_mode='rgb_array', mode=mode)

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


# Training Agent with Q-Learning

In [5]:
action_space_size = env.action_space.n
print(action_space_size)
state_space_size = env.observation_space
print(state_space_size)


6
Box(0, 255, (210, 160, 3), uint8)


The env.observation_space is of type Box, which represents a continuous observation space. It is described as Box(0, 255, (210, 160, 3), uint8), indicating that the observations are represented as a 3-dimensional array of shape (210, 160, 3), where each element is an 8-bit unsigned integer (uint8) ranging from 0 to 255.

Since the observation space is continuous and not discrete, it does not have a fixed number of possible states represented by the attribute n. Instead, the observation values can take any real value within the defined range and shape.

If your state space is continuous, it is not practical to create a Q-table, as it would require an excessive amount of memory to store values for every possible state. In such cases, you typically use function approximation methods, such as neural networks, to approximate the Q-values or the value function.

To solve this we need to lower the observation space, i will do this by setting the game to greyscale.

In [6]:
env = GrayscaleObservationV0(env)

In [13]:
action_space_size = env.action_space.n
print(action_space_size)
state_space_size = env.observation_space.shape
print(state_space_size)
q_table = numpy.zeros((state_space_size[0], state_space_size[1], action_space_size))
print(q_table)

6
(210, 160)
[[[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  ...
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  ...
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  ...
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]

 ...

 [[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  ...
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  ...
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  ...
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]]


# Agent Setup (this need to point the trained agent)

In [ ]:
def agent(env):
    """Random agent that samples actions from the environment's action space."""
    return env.action_space.sample()

# Recording Setup

In [ ]:
output_dir = "q-learning_video"
video_output_frequency = 20
env = RecordVideo(env, output_dir, episode_trigger=lambda episode_id: episode_id == 1 or (episode_id != 0 and (episode_id) % video_output_frequency == 0))
env.metadata['render_fps'] = 24

/Users/archit3ct/anaconda3/envs/AI_Mac/lib/python3.10/site-packages/gymnasium/wrappers/record_video.py:87: UserWarning: WARN: Overwriting existing videos at /Users/archit3ct/Code/Assignments/AI_Mandatory_2/Timothy/q-learning_video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


# Testing Agent Performance

In [ ]:
def get_agent_scores(num_episodes = 1):
    scores = pandas.DataFrame(columns=['score'])
    total_reward = 0
    #highest_score = 0
    
    for _ in range(0, num_episodes + 1):
        #if(_ > 1 and total_reward > 0):
        #    print(f'Game Episode {_ - 1}: Score: {total_reward}')
        env.close()
        total_reward = 0
        observation, info = env.reset()  # Getting start stats from the game
        lives = info['lives']  # Initialize lives inside the loop
        while lives != 0:
            # For demonstration purposes, we use a random agent
            action = agent(env)   # Just a random agent
            obs, reward, terminated, truncated, info = env.step(action)
            lives = info['lives']
            total_reward = total_reward + reward

        # Update highest score if current total reward is higher
        #if total_reward > highest_score:
        #    highest_score = total_reward

        scores = pandas.concat([scores, pandas.DataFrame({'score': [total_reward]})], ignore_index=True)


    """ # Print the highest score achieved
    print(f"Highest Score: {highest_score}") """

    # Close the environment to finalize the video recording
    scores = scores.drop(scores.index[0])
    return scores


In [ ]:
num_episodes = 100

scores = get_agent_scores(num_episodes = num_episodes)



Moviepy - Building video /Users/archit3ct/Code/Assignments/AI_Mandatory_2/Timothy/q-learning_video/rl-video-episode-1.mp4.
Moviepy - Writing video /Users/archit3ct/Code/Assignments/AI_Mandatory_2/Timothy/q-learning_video/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /Users/archit3ct/Code/Assignments/AI_Mandatory_2/Timothy/q-learning_video/rl-video-episode-1.mp4


Moviepy - Building video /Users/archit3ct/Code/Assignments/AI_Mandatory_2/Timothy/q-learning_video/rl-video-episode-1.mp4.
Moviepy - Writing video /Users/archit3ct/Code/Assignments/AI_Mandatory_2/Timothy/q-learning_video/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /Users/archit3ct/Code/Assignments/AI_Mandatory_2/Timothy/q-learning_video/rl-video-episode-1.mp4


Moviepy - Building video /Users/archit3ct/Code/Assignments/AI_Mandatory_2/Timothy/q-learning_video/rl-video-episode-20.mp4.
Moviepy - Writing video /Users/archit3ct/Code/Assignments/AI_Mandatory_2/Timothy/q-learning_video/rl-video-episode-20.mp4



Moviepy - Done !
Moviepy - video ready /Users/archit3ct/Code/Assignments/AI_Mandatory_2/Timothy/q-learning_video/rl-video-episode-20.mp4


Moviepy - Building video /Users/archit3ct/Code/Assignments/AI_Mandatory_2/Timothy/q-learning_video/rl-video-episode-20.mp4.
Moviepy - Writing video /Users/archit3ct/Code/Assignments/AI_Mandatory_2/Timothy/q-learning_video/rl-video-episode-20.mp4



Moviepy - Done !
Moviepy - video ready /Users/archit3ct/Code/Assignments/AI_Mandatory_2/Timothy/q-learning_video/rl-video-episode-20.mp4


# Descriptive Statistics of Agent Performance

In [ ]:
scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 1 to 100
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   score   100 non-null    float64
dtypes: float64(1)
memory usage: 932.0 bytes


In [ ]:
scores.head()

,score
1,20.0
2,225.0
3,100.0
4,170.0
5,30.0


In [ ]:
scores.tail()

,score
96,120.0
97,110.0
98,105.0
99,85.0
100,90.0


In [ ]:
scores.describe()

,score
count,100.000000
mean,143.250000
std,113.679007
min,5.000000
25%,65.000000
50%,120.000000
75%,172.500000
max,715.000000


In [ ]:

index_start = 0
index_end = num_episodes-1

print(f"Game Episode 1: Score {scores.iloc[index_start]['score']}\nGame Episode {num_episodes}: Score {scores.iloc[index_end]['score']}")

Game Episode 1: Score 20.0
Game Episode 100: Score 90.0


In [ ]:
print(f"Lowest Score: {scores['score'].min()}\nHigh Score: {scores['score'].max()}")

## Mode (most frequent element)
It is possible for a sample to have multiple modes. In statistics, a mode refers to the value or values in a dataset that occur most frequently. If there are multiple values with the same highest frequency, the dataset is considered multimodal.

In [ ]:
scores.mode()

,score
0,120.0


## Median (middle value of the data set)

In [ ]:
scores.median()

score    120.0
dtype: float64

## Mean (average)

In [ ]:
scores.mean()

score    137.65
dtype: float64

## Variance (Dissimilarity between samples)

For samples degrees of freedom (ddof) is 0.

Note: The number is high due there is a lot of Dissimilarities between each score.

$$\sigma=\frac{\sum\left(x_i-\mu\right)^2}{n}\text{ for samples (degrees of freedom is 0)}$$ 

In [ ]:
scores.var(ddof=0)

score    7426.2275
dtype: float64

## Standard Deviation ( variability within a sample)
For samples degrees of freedom (ddof) is 0.

$$SD_0=\sqrt{\frac{\sum\left(x_i-\mu\right)^2}{n}}\text{ for samples (degrees of freedom is 0)}$$

In [ ]:

scores.std(ddof=0)

score    86.175562
dtype: float64

# Report Generation

In [ ]:
report = pandas_profiling.ProfileReport(scores).to_file('q-learning_agent_scores_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]